# Compare tweets and party programs

https://github.com/adsieg/text_similarity/blob/master/Different%20Embeddings%20%2B%20Cosine%20Similarity%20%2B%20HeatMap%20illustration.ipynb 

https://www.machinelearningplus.com/nlp/cosine-similarity/


In [34]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime 
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import gensim
from gensim.models import Word2Vec

In [20]:
programme = open('party_programmes/gruene.txt', "r").read()

programme = programme.strip()

In [16]:
for_comp = [programme, "Karl Lauterbach soll Gesundheitsminister werden.", "Klimawandel ist ein wichtiges Thema"]

In [56]:
# Create the Document Term Matrix
#stole this code from here: https://www.machinelearningplus.com/nlp/cosine-similarity/  


count_vectorizer = CountVectorizer() ## need to remove stopwords and use TfidfVectorizer instead?? 
sparse_matrix = count_vectorizer.fit_transform(for_comp)

# OPTIONAL: Convert Sparse Matrix to Pandas Dataframe if you want to see the word frequencies.
doc_term_matrix = sparse_matrix.todense()
df = pd.DataFrame(doc_term_matrix, 
                  columns=count_vectorizer.get_feature_names(), 
                  index=['programme', 'Lauterbach', 'Klima'])
df


,000,030,10,100,101,10115,102,103,104,105,...,überzeugte,überzeugung,überzugehen,üblichen,übrig,übrige,übten,übung,übungsleiter,ülkücü
programme,18,2,26,22,6,2,5,8,5,9,...,1,2,1,1,1,1,1,1,1,1
Lauterbach,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Klima,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity(df, df))

[[1.         0.06450307 0.06484987]
 [0.06450307 1.         0.        ]
 [0.06484987 0.         1.        ]]


In [39]:
# get trained model

# downloaded a German word embedding model from http://devmount.github.io/GermanWordEmbeddings/#download
model = gensim.models.KeyedVectors.load_word2vec_format("/Users/johannes/Downloads/german.model", binary=True) # i know you guys didn't download it yet but just testing things. 

In [58]:
model.cosine_similarities(np.asarray(df[0].values), np.asarray(df[1].values))

KeyError: 0